#  DiVerAI


You are a consultant from DiVerAI and you have just signed a contract with a publishing company. With the rise of GenAI, a lot of people have started to write their own books and now want to publish them. The problem of this publishing company is that their editors don’t have the time to read all of those books. That’s why you were charged to find a way to:   


*   summarize the books content
* analyze the genre of the book
* help the editor to decide if it’s worth reading and publishing


Use the principles and techniques you’ve seen in your quickstart notebook. For this challenge you have to deliver an application using Gradio that can summarize all the text inputs it will be given as well as classify its genre.    
You have to use a LLama model, per your client’s request, since they have heard it’s the best performing model on the market. Therefore you can’t use any OpenAI models, but you are allowed to use Auto-GPTQ.


### import and install

In [ ]:
!pip install transformers torch accelerate gradio langchain-community langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer , pipeline
import transformers
import torch
import gradio as gr
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import HuggingFacePipeline


initilzing the model

In [ ]:
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

###create pipline

In [ ]:
llama2_pipeline = pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",

    max_new_tokens=50,
    do_sample=True,
    temperature=0.3,
    top_p=0.9,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
llm = HuggingFacePipeline(pipeline=llama2_pipeline)


<ipython-input-5-e090f3015ee5>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llama2_pipeline)


crearte prompt template

In [ ]:
summary_prompt =PromptTemplate(
      input_variables=["book_content"],
    template="""
You are a skilled book summarizer. Given the book content below, provide a concise summary (100 words or fewer) without restating the prompt.

Book Content: {book_content}

Summary:""")

In [ ]:
genre_prompt = PromptTemplate(
      input_variables=["book_content"],
    template="""
You are an expert in summarizing books by genre. Carefully analyze the provided book content and generate a concise summary tailored to its genre without additional remarks or instructions.

Book Content: '{book_content}'

Genre (only the label):""")

In [ ]:
editor_prompt = PromptTemplate(
      input_variables=["book_content"],
    template="""
You are an experienced editor evaluating a book for potential publication. Analyze the provided book content and assess whether it is worth reading and publishing. Focus on aspects such as originality, clarity, engagement, and market potential.

Book Genre: '{book_genre}'
Book Content: '{book_content}'

Evaluation:
1. Originality:
2. Clarity and coherence:
3. Engagement and readability:
4. Market potential:

Final Recommendation (Publish, Revise, Reject): """)

In [ ]:
print(editor_prompt.invoke({"book_content": "This is a sample book content."}))


text="\nYou are an experienced editor evaluating a book for potential publication. Analyze the provided book content and assess whether it is worth reading and publishing. Focus on aspects such as originality, clarity, engagement, and market potential.\n\nBook Content: 'This is a sample book content.'\n\nEvaluation:\n1. Originality:\n2. Clarity and coherence:\n3. Engagement and readability:\n4. Market potential:\n\nFinal Recommendation (Publish, Revise, Reject): "


create chains

In [ ]:
summary_chain = LLMChain(llm=llm, prompt=summary_prompt)
genre_chain = LLMChain(llm=llm, prompt=genre_prompt)
editor_chain = LLMChain(llm=llm, prompt=editor_prompt)

<ipython-input-9-e6859361504b>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summary_chain = LLMChain(llm=llm, prompt=summary_prompt)


create example of book to test

In [ ]:
book_sample = "Meta, formerly known as Facebook, has transformed the digital landscape by pioneering advancements in social connectivity, artificial intelligence, and the metaverse. With platforms like Facebook, Instagram, and WhatsApp under its umbrella, Meta has reshaped how people communicate, share content, and engage with digital experiences. The company's ambitious push into the metaverse aims to create immersive virtual spaces where users can interact, work, and socialize using augmented and virtual reality technologies. Despite facing challenges related to privacy concerns and regulatory scrutiny, Meta continues to drive innovation, shaping the future of digital interactions and artificial intelligence-driven solutions."

In [ ]:
test_summary = summary_chain.run(book_content=book_sample)


In [ ]:
print(test_summary)


You are a skilled book summarizer. Carefully analyze the provided book content and generate a concise summary without additional remarks or instructions.

Book Content: Meta, formerly known as Facebook, has transformed the digital landscape by pioneering advancements in social connectivity

Summary (limited to 100 words or fewer): 
In this book, the author explores how Meta, previously known as Facebook, revolutionized online interaction through innovations that reshaped the digital world. From its early days as a small startup to its current status as a global leader, the


In [ ]:
def eval_book(book_content):
    summary = summary_chain.run(book_content=book_content)
    genre = genre_chain.run(book_content=book_content)
    evaluation = editor_chain.run(book_content=book_content)

    summary = summary.split("Summary:")[-1].strip()
    genre = genre.split("Genre (only the label):")[-1].strip()
    evaluation = evaluation.split("Evaluation:")[-1].strip()

    return summary, genre, evaluation

In [ ]:
eval_book(book_sample)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


("In this article, the author discusses Meta's impact on society through its various platforms such as Facebook, Instagram, and WhatsApp. They also explore the company's foray into the metaverse and its potential implications on digital",
 'Technology/Business',
 "1. Originality:\n2. Clarity and coherence:\n3. Engagement and readability:\n4. Market potential:\n\nFinal Recommendation : ❓\n\nAs an editor, I thoroughly analyzed the book content you provided, focusing on various aspects that determine the quality and potential success of a published work. Here's my evaluation based on four key factors: originality,")

### gradio

In [ ]:
def gradio_eval_book(book_content):
   summary, genre, evaluation = eval_book(book_content)
   return {
       "Summary": summary,
       "Genre": genre,
       "Evaluation": evaluation
   }

In [ ]:
interface = gr.Interface(
    fn=gradio_eval_book,
    inputs=gr.Textbox(lines=10, label="Enter Book Content"),
    outputs=gr.Textbox(label="Analysis Results"),
    title="Book Content Analyzer",
    description="Analyze book content to get summary, genre and editorial evaluation"
)
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://24601ec337881bf059.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
